# Finetune Models

 ## Tune "all-MiniLM-L6-v2" using CosineSimilarityLoss

In [1]:
import random
import torch
import numpy as np
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

def set_seed(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def create_pos_neg_pairs(texts, labels, num_samples=5000):
    label_to_texts = {}
    for text, label in zip(texts, labels):
        label_to_texts.setdefault(label, []).append(text)

    train_examples = []
    unique_labels = list(label_to_texts.keys())

    for _ in range(num_samples):
        pos_label = random.choice(unique_labels)
        neg_label = random.choice([l for l in unique_labels if l != pos_label])

        if len(label_to_texts[pos_label]) < 2:
            continue  # not enough positives

        pos_pair = random.sample(label_to_texts[pos_label], 2)
        neg_sample = random.choice(label_to_texts[neg_label])

        train_examples.append(InputExample(texts=pos_pair, label=1.0))  # Positive
        train_examples.append(InputExample(texts=[pos_pair[0], neg_sample], label=0.0))  # Negative

    return train_examples

#  Set seed
set_seed(42)

#  Load TREC dataset
from data.trec_data import TrecDataset
dataset = TrecDataset()
(train_x, train_y), _, _ = dataset.get_splits()

#  Create training data
train_examples = create_pos_neg_pairs(train_x, train_y, num_samples=5000)
train_loader = DataLoader(train_examples, shuffle=True, batch_size=16)

#  Load SBERT model
model = SentenceTransformer("all-MiniLM-L6-v2")

#  Use CosineSimilarityLoss (float labels required)
train_loss = losses.CosineSimilarityLoss(model)

#  Fine-tune
model.fit(
    train_objectives=[(train_loader, train_loss)],
    epochs=1,
    warmup_steps=100,
    weight_decay=0.01,
    optimizer_params={'lr': 2e-5},
    max_grad_norm=0.5,
    show_progress_bar=True
)

#  Save the model
model.save("models/sbert-trec-cosine")
print(" Fine-tuned and saved models/sbert-trec-cosine")


C:\Users\Arun.Tipingiri\AppData\Local\miniconda3\envs\gp_7643\Lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/625 [00:00<?, ?it/s]

 Fine-tuned and saved models/sbert-trec-cosine


## Tune "distilbert-base-nli-stsb-mean-tokens" using CosineSimilarityLoss

In [2]:
import random
import torch
import numpy as np
from sentence_transformers import SentenceTransformer, InputExample, losses
from torch.utils.data import DataLoader

def set_seed(seed=42):
    np.random.seed(seed)
    random.seed(seed)
    torch.manual_seed(seed)
    if torch.cuda.is_available():
        torch.cuda.manual_seed_all(seed)

def create_pos_neg_pairs(texts, labels, num_samples=5000):
    label_to_texts = {}
    for text, label in zip(texts, labels):
        label_to_texts.setdefault(label, []).append(text)

    train_examples = []
    unique_labels = list(label_to_texts.keys())

    for _ in range(num_samples):
        pos_label = random.choice(unique_labels)
        neg_label = random.choice([l for l in unique_labels if l != pos_label])

        if len(label_to_texts[pos_label]) < 2:
            continue

        pos_pair = random.sample(label_to_texts[pos_label], 2)
        neg_sample = random.choice(label_to_texts[neg_label])

        train_examples.append(InputExample(texts=pos_pair, label=1.0))               #  positive
        train_examples.append(InputExample(texts=[pos_pair[0], neg_sample], label=0.0))  #  negative

    return train_examples

#  Set seed
set_seed(42)

#  Load TREC dataset
from data.trec_data import TrecDataset
dataset = TrecDataset()
(train_x, train_y), _, _ = dataset.get_splits()

#  Create training pairs
train_examples = create_pos_neg_pairs(train_x, train_y, num_samples=5000)
train_loader = DataLoader(train_examples, shuffle=True, batch_size=16)

#  Load model
model = SentenceTransformer("distilbert-base-nli-stsb-mean-tokens")

#  Cosine similarity loss (float labels)
train_loss = losses.CosineSimilarityLoss(model)

#  Fine-tune
model.fit(
    train_objectives=[(train_loader, train_loss)],
    epochs=1,
    warmup_steps=100,
    weight_decay=0.01,
    optimizer_params={'lr': 2e-5},
    max_grad_norm=0.5,
    show_progress_bar=True
)

#  Save fine-tuned model
model.save("models/distilbert-trec-cosine")
print(" Fine-tuned and saved: distilbert-trec-cosine")


Found existing file: TREC_Dataset\train_5500.label
Found existing file: TREC_Dataset\TREC_10.label


C:\Users\Arun.Tipingiri\AppData\Local\miniconda3\envs\gp_7643\Lib\site-packages\huggingface_hub\file_download.py:797: FutureWarning: `resume_download` is deprecated and will be removed in version 1.0.0. Downloads always resume when possible. If you want to force a new download, use `force_download=True`.
  warnings.warn(


Epoch:   0%|          | 0/1 [00:00<?, ?it/s]

Iteration:   0%|          | 0/625 [00:00<?, ?it/s]

 Fine-tuned and saved: distilbert-trec-cosine
